In [239]:
# !python3 -m pip install matplotlib
# !python3 -m pip install scipy

In [240]:
import xarray as xr
import matplotlib
import csv
import dask
import parflow as pf
import pandas as pd
import numpy as np

In [241]:
def apply_mask(df, mask):
    return np.where(mask==0, np.nan, df)

In [242]:
RUN_DIR = "/Users/ben/Documents/GitHub/parflow-models/code/models/east_taylor2/simulations/WY2003_predictor_training_save"
RUN_NAME = "EastTaylor_2003"

In [243]:
variables = [
    "pressure", "pressure_becomes_negative", "pressure_becomes_positive", "saturation",
    "porosity", "specific_storage", 
    "permeability_x", "permeability_y","permeability_z"
    ]
num_timesteps = 8760-1
time_steps_to_capture = range(0, num_timesteps)
run_location = f"{RUN_DIR}/{RUN_NAME}.pfidb"
unprocessed_run = pf.Run.from_definition(run_location).data_accessor
# Resolution
dx = unprocessed_run.dx
dy = unprocessed_run.dy
# Thickness of each layer, bottom to top
dz = unprocessed_run.dz

# Extent
nx = unprocessed_run.shape[2]
ny = unprocessed_run.shape[1]
nz = unprocessed_run.shape[0]
BOUNDARY_PRESSURE = 0
# ------------------------------------------
# Time-invariant values
# ------------------------------------------
porosity = unprocessed_run.computed_porosity
specific_storage = unprocessed_run.specific_storage
mask = unprocessed_run.mask
# et = run.et                        # shape (nz, ny, nx) - units 1/T.
slope_x = unprocessed_run.slope_x               # shape (ny, nx)
slope_y = unprocessed_run.slope_y               # shape (ny, nx)
mannings = unprocessed_run.mannings    
# chage this to some type of array with dataframe as each entry
processed_run = pd.DataFrame(
    index=time_steps_to_capture, 
    columns=variables
)
for time in time_steps_to_capture:
    unprocessed_run.time = time
    pressure = apply_mask(unprocessed_run.pressure, mask)
    saturation = apply_mask(unprocessed_run.saturation, mask)
    pressure_is_negative = np.where(pressure < BOUNDARY_PRESSURE,1,0)
    pressure_is_positive = np.where(pressure > BOUNDARY_PRESSURE,1,0)
    processed_run["pressure"][time] = pressure
    unprocessed_run.time = time + 1
    next_pressure = apply_mask(unprocessed_run.pressure, mask)
    next_pressure_is_negative = np.where(next_pressure < BOUNDARY_PRESSURE,1,0)
    next_pressure_is_positive = np.where(next_pressure > BOUNDARY_PRESSURE,1,0)
    pressure_becomes_positive = pressure_is_negative * next_pressure_is_positive
    pressure_becomes_negative = pressure_is_positive * next_pressure_is_negative
    processed_run["pressure_becomes_positive"][time] = pressure_becomes_positive
    processed_run.loc[time] = pd.Series({
        "pressure": pressure,
        "pressure_becomes_positive": pressure_becomes_positive,
        "pressure_becomes_negative": pressure_becomes_negative,
        "saturation" : saturation,
        "mannings": mannings,
        "porosity": unprocessed_run.computed_porosity,
        "specific_storage": unprocessed_run.specific_storage,
        "permeability_x": unprocessed_run.computed_permeability_x, 
        "permeability_y": unprocessed_run.computed_permeability_y, 
        "permeability_z": unprocessed_run.computed_permeability_z, 
        })


Solver: Field BinaryOutDir is not part of the expected schema <class 'parflow.tools.database.generated.Solver'>
  - nt
  - sw_ini
  - qflx_tran_vegm
  - hkdepth
  - wtfact
  - trsmx0
  - smpmax
  - pondmx


In [244]:

run = processed_run
run.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759 entries, 0 to 8758
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   pressure                   8759 non-null   object
 1   pressure_becomes_negative  8759 non-null   object
 2   pressure_becomes_positive  8759 non-null   object
 3   saturation                 8759 non-null   object
 4   porosity                   8759 non-null   object
 5   specific_storage           8759 non-null   object
 6   permeability_x             8759 non-null   object
 7   permeability_y             8759 non-null   object
 8   permeability_z             8759 non-null   object
dtypes: object(9)
memory usage: 616.0+ KB


In [245]:
import plotly.express as px

px.imshow(run.pressure[9][9,:,:])

In [246]:
def times_with_pressure_changes(run, BOUNDARY_PRESSURE=0.0):
    for time in run.index:
        if np.sum(run.pressure_becomes_positive[time][9,:,:]) > 0:
            yield int(time)

print(list(times_with_pressure_changes(run)))

[0, 1, 2, 3, 4, 5, 6, 7, 10, 13, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 34, 36, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 63, 64, 65, 67, 69, 70, 71, 72, 74, 75, 83, 85, 90, 91, 92, 93, 94, 95, 96, 99, 101, 102, 104, 105, 108, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 132, 135, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 184, 185, 186, 187, 188, 189, 192, 193, 194, 195, 196, 197, 199, 200, 201, 203, 204, 207, 208, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 252, 257, 258, 259, 260, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 281, 282, 283, 284, 285, 286, 288, 289, 290, 291, 292, 293, 2

In [247]:
def times_with_pressure_changes(run, BOUNDARY_PRESSURE=0.0):
    for time in run.index:
        if np.sum(run.pressure_becomes_negative[time][9,:,:]) > 0:
            yield int(time)

print(list(times_with_pressure_changes(run)))

[1, 16, 17, 18, 19, 20, 21, 22, 35, 40, 41, 42, 43, 44, 45, 46, 47, 53, 61, 63, 67, 68, 69, 70, 73, 78, 80, 81, 83, 84, 85, 86, 88, 89, 90, 92, 93, 94, 96, 103, 104, 106, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 120, 123, 124, 125, 127, 130, 131, 132, 134, 136, 137, 138, 139, 140, 141, 142, 143, 158, 159, 160, 161, 162, 163, 164, 165, 166, 183, 184, 185, 186, 187, 188, 189, 203, 204, 205, 206, 208, 209, 210, 211, 212, 213, 214, 215, 228, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 251, 252, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 298, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 321, 322, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 343, 345, 347, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 362, 371, 374, 375, 376, 377, 378, 379, 380, 381, 382, 389, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 409, 414, 420, 422, 423, 424, 425, 426, 427, 428, 429, 430, 435, 438, 444,

In [248]:
# # df = pd.read_csv("LW_CLM_Ex3_save/predictor_training_output.csv")

# import linecache

# from pandas import DataFrame

# filename = "LW_CLM_Ex3_short_run/predictor_training_output.csv"
# indices = range(1,3)

# li = []
# for i in indices:
#     li.append(linecache.getline(filename, i).rstrip().split(','))

# print(li[0])
# df2 = DataFrame(li)

In [249]:


#TODO do this for all the processors
processors = range(1,16)
data = pd.read_csv(f"{RUN_DIR}/predictor_training_output.csv.00000")
for processor in processors:
    timestamp = f"00000{processor}"[-5:]
    filename = f"{RUN_DIR}/predictor_training_output.csv.{timestamp}"
    tmp_data = pd.read_csv(filename)
    data = pd.concat([data,tmp_data])
    # df.info()
data = data.rename(columns={"i":"x", "j":"y", "pressure":"runtime_pressure"})
printed_data = data #.set_index(["time", "y", "x"])
printed_data = printed_data.drop_duplicates()

duplicates = printed_data[printed_data.duplicated()]
if len(duplicates)>0:
    print("we had to drop some duplicates...")

printed_data = printed_data.drop_duplicates(subset=["time", "y", "x"], keep="first")
printed_data = printed_data.set_index(["time", "y", "x"]).to_xarray()
# printed_data["time" == 1]
# printed_data = printed_data.sel(time=range(1,num_timesteps+1))


In [250]:
# TODO figure out for sure if we get a duplicate when we change dt
# duplicates = printed_data[printed_data.duplicated(subset=["time", "y", "x"])]
# duplicates
# sel = printed_data.loc[printed_data['time'] == 5592.0]
# sel = sel.loc[sel['x'] == 16]
# sel = sel.loc[sel['y'] == 11]
# sel
# len(printed_data.loc[printed_data['time'] == 1.0])

In [251]:
printed_data

<xarray.Dataset>
Dimensions:             (time: 8761, y: 50, x: 66)
Coordinates:
  * time                (time) float64 1.0 2.0 3.0 ... 8.759e+03 8.76e+03
  * y                   (y) int64 1 2 3 4 5 6 7 8 9 ... 43 44 45 46 47 48 49 50
  * x                   (x) int64 1 2 3 4 5 6 7 8 9 ... 59 60 61 62 63 64 65 66
Data variables:
    volume              (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    max_volume          (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    flux_in             (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    runtime_pressure    (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    we_predicted        (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    predicted_pressure  (time, y, x) float64 nan nan nan nan ... nan nan nan nan

In [252]:
datasets = []
for variable in variables:
    series = np.stack(processed_run[variable])
    run_data = xr.DataArray(
        series, 
        coords={'z':range(0,nz), 'y':range(0,ny),'x': range(0,nx)}, 
        dims=["time", "z", "y", "x"]
    )
    run_data = run_data.to_dataset(name=variable)
    datasets.append(run_data)
datasets.append(printed_data)

In [253]:
merged_data = xr.merge(datasets).sel(z=nz-1)
mannings = xr.DataArray(
        series, 
        coords={'z':range(0,nz), 'y':range(0,ny),'x': range(0,nx)}, 
        dims=["time", "z", "y", "x"]
    ).to_dataset(name="mannings")
merged_data = xr.merge([merged_data, mannings])

px.imshow(merged_data.sel(time=1).pressure - merged_data.sel(time=1).runtime_pressure)



ValueError: cannot reindex or align along dimension 'time' because of conflicting dimension sizes: {8761, 8759} (note: an index is found along that dimension with size=8761)

In [ ]:
#TODO extend above to all the other variables we need
merged_data.to_netcdf(f"east_taylor_data.nc")